In [ ]:
from delta import *
data = spark.range(0, 5)
data.write.format("delta").mode("overwrite").save("/data/lake/numbers")

: 

In [ ]:
df = spark.read.format("delta").load("/data/lake/numbers")
df.show()

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, "/data/lake/numbers")

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 0).load("/data/lake/numbers")
df.show()